In [ ]:
# First Import the Necessary Python Libraries
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import datetime as dt
import gdown


# Replace with your Google Drive file ID
eraurl = f'https://drive.google.com/uc?id=1POHSF4RNhZ496Pa-0DAmPk3wNhtC0As9'
erafile = 'ERA5_temperature.model_level_137.daily.2010-2019.nc.2deg.nc'  # Desired output filename

# Download the file
gdown.download(eraurl, erafile, quiet=False)

erads = xr.open_mfdataset(erafile)


In [ ]:
# Now look at what's inside the dataset
erads

In [ ]:
# The data is 3-dimensional, time x latitude x longitude. The time dimension is 3650 long (365 days x 10 years + 2 leap years in there, 2012 and 2016). Lat is 90, lon is 144.
# The variable in this dataset is near-surface temperature, 't' (in this case lowest model level temperature).
# let's specify variables for the temperature, lat, and lon

eratemp=erads.t # load in the temperature
eratemp= eratemp.sel(time=~((eratemp.time.dt.month == 2) & (eratemp.time.dt.day == 29))) # take out the leap days to make the length of each year the same and make the calculations easier
eratemp=1.8*(eratemp-273.15)+32 # convert to Fahrenheit
eratemp=eratemp.chunk({'time':100}) # convert to dask chunks so the binder can handle the data better
eralon=erads.lon # load in the longitude
eralat=erads.lat # load in the latitude
eratime=erads.time # load in the time

In [ ]:
# Okay, now let's plot the first day in the data set, January 2nd 2010.

eratemp_want=eratemp.sel(time='2010-01-02')

# specify our colorbar properties
color_min = -30 # lower limit of -40 degrees F
color_max = 110 # upper limit of 120 degrees 5
numberofsteps=15
colorlevels=np.linspace(color_min, color_max, numberofsteps)

# Initialize the figure with the specified projection
fig = plt.figure(figsize=(36, 6))
# We also have to specify a map projection, projection=ccrs.PlateCarree(central_longitude=0)
axs = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree(central_longitude=0))

eratemp_want_plot = axs.contourf(eralon, eralat, eratemp_want,levels=colorlevels,extend='both')

# Add coastlines to the plot
axs.coastlines(resolution='110m')
axs.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

# Add a colorbar with a label containing the degree symbol and adjust font size
cbar = fig.colorbar(eratemp_want_plot)
cbar.set_label('°F', fontsize=16)

# Add a title to the plot
plt.title('Temperature '+pd.Timestamp(eratemp_want.time.values).strftime('%Y-%m-%d'))

# Show the plot
plt.show()


In [ ]:
#Answer the following questions to check for understanding, you can type it right into the notebook

#1. What colors do you see near the equator? What about near the poles? Does this make sense according to the colorbar?

#2. Where is it the coldest?

#3. Which is warmer, the Northern Hemisphere or the Southern Hemisphere? Why?



In [ ]:
# What's next is some tasks to practice your coding skills. It's probably best to copy and paste some of the code above and modify where needed.

#1. Plot the temperature for January 3rd, 2010

In [ ]:
#2. Plot the temperature for May 3rd, 2010

In [ ]:
#3. Plot the temperature for June 2nd, 2013

In [ ]:
#4. Plot the temperature for July 25, 2013

In [ ]:
#5. Plot the temperature for your birthday or someone you love or admire's birthday in 2017

In [ ]:
# Critical thinking:
# 1. Look at your plots for January and August. How do the Northern and Southern Hemisphere's compare in both of these months? What about in May? Does this all make sense?

In [ ]:
# Now let's plot the average temperature of the whole dataset. Another way of saying this is the climatology of the annual mean

climannual=eratemp.mean(dim='time')

# Initialize the figure with the specified projection
fig = plt.figure(figsize=(36, 6))
# We also have to specify a map projection, projection=ccrs.PlateCarree(central_longitude=0)
axs = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree(central_longitude=0))

eratemp_want_plot = axs.contourf(eralon, eralat, climannual,levels=colorlevels,extend='both')

# Add coastlines to the plot
axs.coastlines(resolution='110m')
axs.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

# Add a colorbar with a label containing the degree symbol and adjust font size
cbar = fig.colorbar(eratemp_want_plot)
cbar.set_label('°F', fontsize=16)

# Add a title to the plot
plt.title('Annual Mean Temperature 2010-2019')

# Show the plot
plt.show()


In [ ]:
# Observations:
# 1. How does this annual mean temperature compare to the temperature on January 2nd 2010 and the other dates?

In [ ]:
# Now let's plot the average temperature for just Northern Hemisphere (boreal) summer, defined as June, July, August.


climjja=eratemp.sel(time=eratemp['time'].dt.month.isin([6,7,8])).mean(dim='time')

# Initialize the figure with the specified projection
fig = plt.figure(figsize=(36, 6))
# We also have to specify a map projection, projection=ccrs.PlateCarree(central_longitude=0)
axs = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree(central_longitude=0))

eratemp_want_plot = axs.contourf(eralon, eralat, climjja,levels=colorlevels,extend='both')

# Add coastlines to the plot
axs.coastlines(resolution='110m')
axs.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

# Add a colorbar with a label containing the degree symbol and adjust font size
cbar = fig.colorbar(eratemp_want_plot)
cbar.set_label('°F', fontsize=16)

# Add a title to the plot
plt.title('JJA Mean Temperature 2010-2019')

# Show the plot
plt.show()


In [ ]:
# Do the same but for Boreal Winter: December, January, February. How does DJF compare to the annual mean and JJA?

In [ ]:
# Now we will calculate the anomaly which is the difference from the average. This is essentially of how abnormally warm or cold or neither a given day is.
# Let's calculate the anomaly for June 3rd 2016
eratemp20160602=eratemp.sel(time='2016-06-02')

eratemp20160602_anom=eratemp20160602-climannual


In [ ]:
# Plot the raw temperature for that day

# Initialize the figure with the specified projection
fig = plt.figure(figsize=(36, 6))
# We also have to specify a map projection, projection=ccrs.PlateCarree(central_longitude=0)
axs = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree(central_longitude=0))

eratemp_want_plot = axs.contourf(eralon, eralat, eratemp20160602,levels=colorlevels,extend='both')

# Add coastlines to the plot
axs.coastlines(resolution='110m')
axs.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

# Add a colorbar with a label containing the degree symbol and adjust font size
cbar = fig.colorbar(eratemp_want_plot)
cbar.set_label('°F', fontsize=16)

# Add a title to the plot
plt.title('Temperature '+pd.Timestamp(eratemp20160602.time.values).strftime('%Y-%m-%d'))

# Show the plot
plt.show()


In [ ]:
# Plot the temperature anomaly for that day
# specify our colorbar properties
color_minanom = -40 # lower limit
color_maxanom = -color_minanom # upper limit
numberofsteps=11
colorlevelsanom=np.linspace(color_minanom, color_maxanom, numberofsteps)

# specify our colorbar properties
# Initialize the figure with the specified projection
fig = plt.figure(figsize=(36, 6))
# We also have to specify a map projection, projection=ccrs.PlateCarree(central_longitude=0)
axs = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree(central_longitude=0))

eratemp_want_plot = axs.contourf(eralon, eralat, eratemp20160602_anom,levels=colorlevelsanom,extend='both',cmap='bwr')

# Add coastlines to the plot
axs.coastlines(resolution='110m')
axs.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

# Add a colorbar with a label containing the degree symbol and adjust font size
cbar = fig.colorbar(eratemp_want_plot)
cbar.set_label('°F', fontsize=16)

# Add a title to the plot
plt.title('Temperature Anomaly '+pd.Timestamp(eratemp20160602.time.values).strftime('%Y-%m-%d'))

# Show the plot
plt.show()


In [ ]:
# Observe:
# 1. Where is it hotter than the annual average temperature? Where is it colder? Does this make sense?
# 2. Where is it most hotter than average?

In [ ]:
# Plot the temperature anomaly as calculated above, but for January 1,st 2016. How does it compare to  the June date?

In [ ]:
"""
One issue with calculating the anomaly this way is that all the days in June are warmer than the annual average in the northern hemisphere because it's summer, whereas
the opposite holds true for winter.

So a better wait to calculate the anomaly for a summer date would be to use the climatology we calculated for summer. However, this is still problematic. The JJA climatology
would be too warm for dates in early June and late August, and too cold for dates in the middle of July (see below).

So it's best to use a mean state to calculate the anomaly using something called a rolling mean. Let's get a feel for what that means by just focusing on the New York gridcell.

Let's start by looking at the summer temperature timeseries and JJA mean

"""

lonwant=360-74 # NYC is 74 West longitude, but the longitude dimension goes from 0 to 360, so we do 360-74
latwant=40.71
eratempnyc=eratemp.sel(lat=latwant,lon=lonwant,method='nearest')

fig = plt.figure(figsize=(50, 10))

plt.plot(eratempnyc['time'].values,eratempnyc.values, label='Raw Temperature') # plot the summer temperature timeseries

plt.axhline(y=eratempnyc.mean(), color='k', linestyle='--', label='Annual Mean Temperature') # plot a horizontal line with the JJA average in NYC

plt.axhline(y=eratempnyc.sel(time=eratemp['time'].dt.month.isin([6,7,8])).mean(), color='g', linestyle='--', label='JJA Mean Temperature') # plot a horizontal line with the JJA average in NYC

plt.legend(loc='upper left')
plt.grid(True)

# Double clicking the figure should help you to zoom in

# Here you can see how each year from 2010 to 2019 compared to one another in NYC. Which year had the lowest peak and which one had the deepest valley?

In [ ]:
"""
The JJA mean is warmer than dates in early June and late August, and colder than dates in the middle of July.
Now let's take a  30-day rolling mean instead to get a mean that contains this seasonal cycle. That way we can remove the seasonal cycle
to calculate the anomaly. What the rolling mean does is at a given day, it takes 15 days before and 15 days after and averages it all together,
then it moves on to the next day.
"""

eratempnyc30=eratempnyc.rolling(time=30, center=True).mean()

fig = plt.figure(figsize=(50, 10))

plt.plot(eratempnyc['time'].values,eratempnyc.values,linewidth=2,color='b',label='Raw Temperature')
plt.plot(eratempnyc['time'].values,eratempnyc30.values,linewidth=2,color='r',label='30 Day Running Mean Raw Temperature')
plt.axhline(y=eratempnyc.mean(), color='k', linestyle='--', label='Annual Mean Temperature') # plot a horizontal line with the JJA average in NYC
plt.axhline(y=eratempnyc.sel(time=eratemp['time'].dt.month.isin([6,7,8])).mean(), color='g', linestyle='--', label='JJA Mean Temperature') # plot a horizontal line with the JJA average in NYC


# Add labels and title
plt.xlabel('Summer Day')
plt.ylabel('Temperature (F)')
plt.title('New York City')
plt.legend(loc='upper left')
plt.grid(True)

# The red line is the 30 day rolling mean, note that unlike the JJA mean that is flat, this rolling mean captures the seasonal cycle (JJA peaks mid july)

# Notice also, how some summers aren't quite as warm as others (i.e., 2014)

In [ ]:
# So yes, some summers are warmer than others overall, same goes for winter and the other seasons, so after taking the rolling mean, we next want to average all the years together

eratempnyc30_rsnp=eratempnyc30.values.reshape((10,365)) # reshape the time dimension so instead of consecutive days, it is two dimensions, one for year and one for day of the year
eratempnyc30_rs= xr.DataArray(data=eratempnyc30_rsnp,dims=["year","day"],coords=dict(year=np.arange(2010,2020,1),day=np.arange(1,366,1))) #turn back into x-array dataset
dailyclimnyc_rs=eratempnyc30_rs.mean(dim='year') # take the mean over all 10 years, now you have a mean state (or climatology) for each day of the year
dailyclimnyc_np=np.tile(dailyclimnyc_rs.values,10) # repeat this 365 day climatology for 10 years
dailyclimnyc=xr.DataArray(data=dailyclimnyc_np,dims=["time"],coords=dict(time=eratemp.time.values)) # turn the 10 year long cyclic climatology into an xarray dataset

eratempnyc_anom=eratempnyc-dailyclimnyc # calculate the anomaly


In [ ]:
# Plot the temperature anomaly now that it's calculated in a more exact way.
# Let's just plot the first 3 years so we can see more clearly:
# #[:365*3] means select the first index all the way up to the 365*3 index

# Create the plot
fig, ax1 = plt.subplots(figsize=(50, 10))

# Plot the summer temperature timeseries and the 30-day rolling mean
ax1.plot(eratempnyc['time'].values[:365*3], eratempnyc.values[:365*3], linewidth=2, color='b', label='Raw Temperature')
ax1.plot(dailyclimnyc['time'].values[:365*3], dailyclimnyc.values[:365*3], linewidth=2, color='r', label='Climatology')
ax1.axhline(y=eratempnyc.mean(), color='k', linestyle='--', label='Mean Temperature')

# Add labels and title to the first y-axis
ax1.set_xlabel('Summer Day')
ax1.set_ylabel('Temperature (F)')
ax1.set_title('New York City')
ax1.legend(loc='upper left')
ax1.grid(True)

# Create a second y-axis to plot the anomaly
ax2 = ax1.twinx()
ax2.plot(eratempnyc_anom['time'].values[0:365*3], eratempnyc_anom.values[0:365*3], linewidth=2, color='g', label='Anomaly')
ax2.set_ylabel('Temperature Anomaly (F)')
ax2.legend(loc='upper right')

In [ ]:
# The green line now show's the anomalies with the seasonal cycle removed. Do you see the anomalies spike and dip where the raw temperature does (blue)
# Do you see if you add red, the climatology, and green, the anomaly, together you get blue, the raw temperature?

# Now let's focus on the anomalies in JJA only
eratempnyc_anomjja=eratempnyc_anom.sel(time=eratemp['time'].dt.month.isin([6,7,8]))

# Plot a histogram to get a sense of how these temperature anomalies are distributed
plt.hist(eratempnyc_anomjja.values.flatten(),bins='auto')
plt.xlabel('Temperature Anomaly (F)')
plt.ylabel('Count')

In [ ]:
# Where is the center of the histogram roughly?

# What is the biggest temperature anomaly? what is the smallest?

# Now let's calculate the 90th percentile, this is the point where 90th percentile is contained before it
thresh90=np.percentile((eratempnyc_anomjja),90)
print('The 90th percentile is '+str(thresh90)+' Degrees Fahrenheit')

# Tasks for you:

#1. Calculate the 10th percentile.
#2. Calculate the 50th percentile.
#3. Calculate the 99th percentile.
#4. Calculate the 1st percentile.

#do your answers make sense according to the histogram?

In [ ]:
# Plot the 90th percentile onto the histogram. The 90th percentile is a common threshold used to identify extreme events.
# In NYC it's when the temperature is at least 5 degrees above average


plt.hist(eratempnyc_anomjja.values.flatten(),bins='auto')
plt.xlabel('Temperature Anomaly (F)')
plt.ylabel('Count')
plt.axvline(thresh90, color='r', linestyle='dashed', linewidth=2,label='90th Percentile')
plt.legend(loc='upper left')
plt.grid(True)

In [ ]:
# Now we identify where in the data (what dates) are above the 90th percentile threshold to be called a heat extreme

eratempnyc_anomjja90idx = np.squeeze(np.where(eratempnyc_anomjja.values > thresh90))
np.shape(eratempnyc_anomjja90idx)
# Does it make sense to you why this one is 92 data points long if there are 920 JJA days all together in the 10 year dataset?


In [ ]:
# We want to next look at what the temperature anomaly looks like for these extreme events. First let's calculate the proper anomaly at all grid points,
# similar to what we did for NYC, but now for all gridpoints

eratemp30=eratemp.rolling(time=30, center=True).mean()
eratemp30_rsnp=eratemp30.values.reshape((10,365,len(eralat),len(eralon))) # reshape the time dimension so instead of consecutive days, it is two dimensions, one for year and one for day of the year
eratemp30_rs= xr.DataArray(data=eratemp30_rsnp,dims=["year","day","lat","lon"],coords=dict(year=np.arange(2010,2020,1),day=np.arange(1,366,1),lat=eralat,lon=eralon)) #turn back into x-array dataset
dailyclim_rs=eratemp30_rs.mean(dim='year') # take the mean over all years
dailyclim_np=np.tile(dailyclim_rs.values,(10,1,1)) # tile the average at each gridpoint 10 times in the time dimension so it's 10 years long
dailyclim=xr.DataArray(data=dailyclim_np,dims=["time","lat","lon"],coords=dict(time=eratemp.time.values,lat=eralat,lon=eralon)) # turn into xarray
eratemp_anom=eratemp-dailyclim # calculate the anomaly
eratemp_anomjja=eratemp_anom.sel(time=eratemp_anom['time'].dt.month.isin([6,7,8])) # just pick out anomalies in JJA


In [ ]:
# Plot the temperature anomaly on June 2nd 2016 now that we have a proper
# anomaly calculation
color_minanom = -20 # lower limit
color_maxanom = -color_minanom # upper limit
numberofsteps=11
colorlevelsanom=np.linspace(color_minanom, color_maxanom, numberofsteps)

# specify our colorbar properties
# Initialize the figure with the specified projection
fig = plt.figure(figsize=(36, 6))
# We also have to specify a map projection, projection=ccrs.PlateCarree(central_longitude=0)
axs = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree(central_longitude=0))

eratemp_want_plot = axs.contourf(eralon, eralat, eratemp_anom.sel(time='2016-06-02'),levels=colorlevelsanom,extend='both',cmap='bwr')

# Add coastlines to the plot
axs.coastlines(resolution='110m')
axs.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

# Add a colorbar with a label containing the degree symbol and adjust font size
cbar = fig.colorbar(eratemp_want_plot)
cbar.set_label('°F', fontsize=16)

# Add a title to the plot
plt.title('Temperature Anomaly '+pd.Timestamp(eratemp20160602.time.values).strftime('%Y-%m-%d'))

# Show the plot
plt.show()


In [ ]:
# How does this anomaly calculation compare with the previous one?
# Where is it hotter than average, and where is it colder than average? Where is
# it the most hotter than average?

In [ ]:
# A composite is an average over some time. In this case, we will composite (or
# average together) the JJA days that had the top 10% of temperature anomalies
# in NYC and plot it.

# Set Colorbar Limits
color_minanom = -10 # lower limit
color_maxanom = -color_minanom # upper limit
numberofsteps=11 # set number of colors
colorlevelsanom=np.linspace(color_minanom, color_maxanom, numberofsteps)

# Use the indices for NYC JJA anomalies that are above the 90th percentile to
# grab the temperature anomalies for Extreme heat days only. Then average them.
compt=eratemp_anomjja.values[eratempnyc_anomjja90idx,:,:].mean(axis=0)

fig=plt.figure(figsize=(25,10))
#plot  composite
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree()) #1,1,1 a 1x1 subplot and the first entry
ax.set_global()
ax.coastlines('110m', alpha=0.9) #alpha is how dark lines are
plotto=ax.contourf(eralon,eralat,compt,levels=colorlevelsanom,extend='both',cmap='bwr')
ax.set_extent([lonwant-60,lonwant+60,20,70],crs=ccrs.PlateCarree())

# Add a colorbar with a label containing the degree symbol and adjust font size
cbar = fig.colorbar(plotto)
cbar.set_label('°F', fontsize=16)

# Add a title to the plot
plt.title('Composite of Temperature Anomaly During 90th Percentile Events in NYC')

In [ ]:
# Describe what you see.

In [ ]:
# Now a challenge for you. Use the example code above to create a composite of temperature anomaly
# during 90th percentile heat extremes in a location of your choosing. When you plot your composite, you may need to lower the colorbar limit to see clearer.

In [ ]:
# Here's a bonus thing to look at if you have time, or couldn't figure out how
# to do a place of your own for the above lesson. Let's now look at a global
# warming, business as usual simulation (SSP5-8.5) from 2090-2099 using GFDL's
# AM4 GCM.

urlssp1 = f'https://drive.google.com/uc?id=1P5JXetsT-PfE_Hj6VMgUScUAo_nz4YQs'
outputssp1 = 'atmos_level.0002010100-0006123123.temp.0998SigmahPa.daily.2deg.nc'  # Desired output filename
# Download the file
gdown.download(urlssp1, outputssp1, quiet=False)

urlssp2 = f'https://drive.google.com/uc?id=1P5iCXmQevEXZ-1DpBfSe-DJTIA6zPcpl'
outputssp2 = 'atmos_level.0007010100-0011123123.temp.0998SigmahPa.daily.2deg.nc'  # Desired output filename

# Download the file
gdown.download(urlssp2, outputssp2, quiet=False)

# Define the path and name of the data you want to load. Note, the wildcard,
# '*', in the filename means takes all years The data used here is from GFDL's
# AM4 model. The forcing conditions are monthly average sea surface temperatures
# from 1980-2014 from observations.
sspds=xr.open_mfdataset('atmos_level.000*.nc')

ssptemp=sspds.temp # load in the temperature
ssptemp=1.8*(ssptemp-273.15)+32 # convert to Fahrenheit
ssptemp=ssptemp.chunk({'time':100}) # convert to dask chunks so the binder can handle the data better
ssplon=sspds.lon # load in the longitude
ssplat=sspds.lat # load in the latitude
ssptempnyc=ssptemp.sel(lat=latwant,lon=lonwant,method='nearest')
ssptempnyc30=ssptempnyc.rolling(time=30, center=True).mean()
ssptempnyc_rsnp=ssptempnyc.values.reshape((10,365)) # reshape the time dimension so instead of consecutive days, it is two dimensions, one for year and one for day of the year
ssptempnyc_rs= xr.DataArray(data=ssptempnyc_rsnp,dims=["year","day"],coords=dict(year=np.arange(2010,2020,1),day=np.arange(1,366,1))) #turn back into x-array dataset

ssptempnyc30_rsnp=ssptempnyc30.values.reshape((10,365))
ssptempnyc30_rs= xr.DataArray(data=ssptempnyc30_rsnp,dims=["year","day"],coords=dict(year=np.arange(2010,2020,1),day=np.arange(1,366,1)))
dailyclimnyc_rs=ssptempnyc30_rs.mean(dim='year')
dailyclimnyc_np=np.tile(dailyclimnyc_rs.values,10)
dailyclimnyc=xr.DataArray(data=dailyclimnyc_np,dims=["time"],coords=dict(time=ssptemp.time.values))

ssptempnyc_anom=ssptempnyc-dailyclimnyc



In [ ]:
# Let's plot the temperature side by side in the warming simulation and ERA and
# the warming simulation



fig = plt.figure(figsize=(50, 10))

plt.plot(eratempnyc['time'].values,eratempnyc.values, label='ERA') # plot the summer temperature timeseries
plt.axhline(y=eratempnyc.sel(time=eratemp['time'].dt.month.isin([6,7,8])).mean(), color='g', linestyle='--', label='ERA JJA Mean Temperature') # plot a horizontal line with the JJA average in NYC

plt.plot(eratempnyc['time'].values,ssptempnyc.values, label='AM4 SSP5-8.5') # plot the summer temperature timeseries
plt.axhline(y=ssptempnyc.sel(time=eratemp['time'].dt.month.isin([6,7,8])).mean(), color='k', linestyle='--', label='AM4 SSP5-8.5 JJA Mean Temperature') # plot a horizontal line with the JJA average in NYC


plt.legend(loc='upper left')
plt.grid(True)

# Let's see how much warmer JJA in SSP is than ERA on average
erajja=eratempnyc.sel(time=eratemp['time'].dt.month.isin([6,7,8])).mean().values
sspjja=ssptempnyc.sel(time=eratemp['time'].dt.month.isin([6,7,8])).mean().values
changenyc=sspjja-erajja
print(changenyc)

In [ ]:
# Now let's focus on the anomalies in JJA only
ssptempnyc_anomjja=ssptempnyc_anom.sel(time=eratemp['time'].dt.month.isin([6,7,8]))

# Plot a histogram to get a sense of how these temperature anomalies are distributed
plt.hist(ssptempnyc_anomjja.values.flatten(),bins='auto')
plt.xlabel('Temperature Anomaly (F)')
plt.ylabel('Count')

In [ ]:
# How does this distribution compare to the one before?

In [ ]:
# Let's plot both on the same axis:

# Plot the first histogram
plt.hist(eratempnyc_anomjja.values.flatten(), bins='auto', alpha=0.5, label='ERA', color='skyblue')

# Plot the second histogram
plt.hist(ssptempnyc_anomjja.values.flatten(), bins='auto', alpha=0.5, label='SSP', color='salmon')

# Add labels and legend
plt.xlabel('JJA Temperature Anomalies (F)')
plt.ylabel('Count')
plt.legend()

# Show the plot
plt.show()


# It looks like the ERA is a bit skinnier except for at the coldest and warmest edges. So the distributions of the anomalies in ERA reanalysis
# and this warming simulation is similar. One thing, however, is that the warming simulation has overall warmer climatology.

In [ ]:
# But, JJA is about 3 degrees warmer by the end of the 21st century for the SSP5-8.5 scenario in NYC. This has implications for what we call extremes

# Let's plot both on the same axis:
fig=plt.figure(figsize=(10,10))

# Plot the first histogram
plt.hist(eratempnyc_anomjja.values.flatten(), bins='auto', alpha=0.5, label='ERA', color='skyblue')

# Plot the second histogram
plt.hist(ssptempnyc_anomjja.values.flatten()+changenyc, bins='auto', alpha=0.5, label='SSP + Mean State Increase', color='salmon')

# Add labels and legend
plt.xlabel('JJA Temperature Anomalies (F)')
plt.ylabel('Count')
plt.axvline(thresh90, color='r', linestyle='dashed', linewidth=2,label='90th Percentile ERA')
plt.legend()
plt.grid(True)
# Show the plot
plt.show()



In [ ]:
# Now when we factor in the overall warming of the climatology

percentile_rank = np.sum(ssptempnyc_anomjja.values.flatten()+changenyc < thresh90) / len(eratempnyc_anomjja) * 100

print(f"The value {thresh90} is at the {percentile_rank}th percentile.")

# This means that in this scenario, a 5 degree anomaly that was considered a heat extreme in the 2010-2019 climate is now
# only 65th percentile when you consider the fact that summers are 3 degrees warmer on average.
# This means only 2 degree anomalies in the warming scenario would be warm enough to be considered a heat extreme from 2010-2019.
# In other words, you go from experiencing these temperatures only 10 % of the time, to now 35 %!
